## init

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from ase.io import read, write
from ase.visualize import view
from ase.visualize.plot import plot_atoms
from ase.build.tools import sort as sort_atoms
from matplotlib import colors as mcolors

# import sys
# sys.path.insert(0, '/home/djrm/git/hetcattoolbox')

from glob import glob
import pandas as pd
import numpy as np
import ast

from ase.build.tools import sort

In [5]:
from autoadsorbate.string_utils import _example_config, _show_ussage, construct_smiles, xx_get_special_symbols
from autoadsorbate.autoadsorbate import Fragment, Surface
from autoadsorbate.Surf import conformer_to_site
from autoadsorbate.utils import get_backbone_bond_change,read_relax_traj,  read_relax_dir, compute_energy, snap_pos_compare
from autoadsorbate.utils import _compare_pos, slice_traj_by_formula,  get_drop_snapped, count_C_next_to_O
from autoadsorbate.plotting import *

In [3]:
import seaborn as sns
import plotly.express as px
import matplotlib.pyplot as plt
import matplotlib.font_manager
plt.rcParams["font.family"] = "Arial"

## read data

In [ ]:
relaxed_traj = read('./collect_data/relaxed_traj.xyz', index=':')
rdf = pd.read_csv('./collect_data/data_rdf.csv')

_tmp = {}
for k in ast.literal_eval(rdf.slab_info.values[0]).keys():
    _tmp[k] = [ast.literal_eval(d)[k] for d in rdf.slab_info.values]
    rdf[k] = _tmp[k]

pid = rdf['mpid'] + '-' + rdf['mi'] + '-' + rdf['iterm'].astype(str) #[f"{ast.literal_eval(d)['mpid']}_{ast.literal_eval(d)['mi']}_{ast.literal_eval(d)['iterm']}" for d in rdf.slab_info.values]
rdf['pid'] = pid

In [ ]:
import json
with open('./collect_data/identifiers.json') as f:
    identifiers = json.load(f)
smiles = [i.split('--')[1] for i in identifiers]

In [ ]:
from mace.calculators import mace_mp

parent_en = {}
for pid in rdf.pid.unique():
    traj_index = rdf[rdf.pid.isin([pid])].traj_index.iloc[0]
    atoms = relaxed_traj[traj_index].copy()
    ref_atoms = atoms[[atom.index for atom in atoms if atoms.arrays['fragments'][atom.index] == 0]]
    calc = mace_mp(model='./models/mace-mp-0b3-medium.model', dispersion=True, device='cpu')
    ref_atoms.set_calculator(calc)
    parent_en[pid] = ref_atoms.get_potential_energy()
    ref_atoms.info={'pid': pid}

ref_dict={
    'C' : 0,
    'O' : 0,
    'H' : 0
}

# parent_en = read('./relax_Cu111_Cu2_try2/MACE_relax_Cu111_Cu2_slab.xyz', index=-1).get_potential_energy()

xdf = compute_energy(rdf, ref_dict, parent_en)
# xdf = xdf[xdf['energy']>-100]

In [ ]:
bkp_xdf = xdf.copy()

## chemiscope

In [ ]:
chemiscope.write_input(
    frames=[center_fragment_in_cell(relaxed_traj[i], fragment_inds=[1]) for i in _xdf.traj_index.values],
    properties=_xdf.to_dict(orient='list'),
    path='chemiscope_input.json'
    # mode='default',
)

### plot atoms

In [ ]:
plot_most_stable(_xdf, relaxed_traj)

In [ ]:
make_hist_plot(_xdf)

In [ ]:
ax, heat_map = plot_energy_heatmap(_xdf,
                    column = 'energy_calibrated',
                    std = 0.05,
                    e_min=-0.1,
                    e_max=3,
                    resolution='auto',
                    normalize=True,
                    return_heatmap=True,
                    T=True,
                    cmap = "Blues"
                    )

In [ ]:
ax, heat_map = plot_energy_heatmap(_xdf,
                    column = 'energy_calibrated',
                    std = 0.05,
                    # e_max=_xdf.energy_calibrated.max()+5*std,
                    # e_min=_xdf.energy_calibrated.min()-5*std,
                    e_min=-0.1,
                    e_max=3,
                    resolution='auto',
                    normalize=False,
                    return_heatmap=True,
                    T=True)

## paper prep

In [ ]:
def plot_energy_heatmap(_xdf, column, std, e_min, e_max, resolution, normalize,
                        return_heatmap=False, T=False, cmap ='viridis', normalize_mode='max', ax=None):
    heat_map = []
    yticklabels = []
    

    for i, backbone in enumerate(_xdf.backbone.unique()):
        for j, H in enumerate(_xdf.H.unique()):
            df_slice = _xdf[_xdf.H.isin([H]) & _xdf.backbone.isin([backbone])]
            if len(df_slice) > 0:
                v = energy_descriptor_from_slice(
                        df_slice,
                        column=column,
                        std=std,
                        e_min=e_min,
                        e_max=e_max,
                        resolution=resolution,
                        normalize=normalize,
                        normalize_mode=normalize_mode
                    )
                heat_map.append(v[1])  
                yticklabels.append(df_slice.calibrate_keys.values[0])
    heat_map = np.array(heat_map)

    xticklabels = []
    wanted_labels = np.arange(-10, 10, 0.4)
    for i, e in enumerate(v[0]):
        if any(np.abs(e - wanted_labels) < 1e-2):
            label = str(np.round(e, 1))
            if label not in xticklabels + ['-0.0']:
                xticklabels.append(label)
            else:
                xticklabels.append('')
            
        else:
            xticklabels.append('')

    if ax == None:
        fig = plt.figure()
        ax=fig.add_subplot(111)
    if T==False:
        sns.heatmap(heat_map, xticklabels=xticklabels, yticklabels=yticklabels, cbar=False, ax=ax)
        for i in range(heat_map.shape[0]+1):
            ax.axhline(i, color='white', lw=2 )
    
    else:
        ax = sns.heatmap(heat_map.T, xticklabels=yticklabels, yticklabels=xticklabels, cbar=False, cmap=cmap, ax=ax)   
        for i in range(heat_map.shape[1]+1):
            ax.axvline(i, color='white', lw=0.5)
    
    ax.tick_params(axis='both', which='both', length=0)
    ax.tick_params(axis='x', labelsize=6)
    ax.tick_params(axis='y', labelsize=6)
    ax.invert_yaxis()
    
    if return_heatmap:
        return heat_map

## Figure SI

In [ ]:
xdf = bkp_xdf.copy()
# xdf = xdf[(xdf.material_formula == 'Cu') & xdf.mi.isin(['1#1#1','1#0#0'])]
_xdf = filter_xdf(xdf, relaxed_traj)

In [ ]:
traj_xdf = [relaxed_traj[i] for i in _xdf.traj_index.values]
write('./traj_xdf_11032025.xyz', traj_xdf)

In [ ]:
font = {'size'   : 7}
matplotlib.rc('font', **font)

fig, axs = plt.subplots(len(_xdf.pid.unique()), 1, figsize=[7.2,5], sharex=True)

for i, pid in enumerate(_xdf.pid.unique()):
    sns.histplot(_xdf[_xdf.pid==pid], x='energy_calibrated', hue='pid', ax=axs[i], binrange=[0,8], binwidth=0.1)

fig.set_layout_engine(layout='tight')
fig

In [ ]:
# fig.savefig('figure_S_histogram.png', dpi=600)

In [ ]:
import itertools

# for mi in rdf.mi.unique()[:1]:
# for metal in rdf.material_formula.unique():
for mi, metal in list(itertools.product(rdf.mi.unique(), rdf.material_formula.unique()))[:1]:
   
    print(mi)

    xdf = bkp_xdf.copy()
    # xdf = xdf[xdf.mi.isin([mi])]
    xdf = xdf[xdf.material_formula == metal]
    # xdf = xdf[(xdf.material_formula == metal) & xdf.mi.isin([mi])]
    _xdf = filter_xdf(xdf, relaxed_traj)

    fig, axs = plt.subplots(
        ncols=len(_xdf.H.unique()),
        nrows=len(_xdf.backbone.unique()),
        figsize=[7.2,8]
    )

    _xdf = _xdf.sort_values(by=['H', 'backbone'])

    view_atoms = []

    for i, backbone in enumerate(_xdf.backbone.unique()):
        for j, H in enumerate(_xdf.H.unique()):

            ax = axs[i,j]
            
            df_slice = _xdf[_xdf.H.isin([H]) & _xdf.backbone.isin([backbone])]
            # df_slice.sort_values(by=['energy', 'backbone', 'H'],ascending=True)
            df_slice=df_slice[df_slice.energy==df_slice.energy.min()]
            
            if len(df_slice) > 0:
                e = np.round(df_slice.iloc[0].energy,2)
                origin = df_slice.iloc[0].origin
                traj_index = df_slice.iloc[0].traj_index
                
                atoms = relaxed_traj[traj_index].copy()
                atoms_center = get_fragment_center(atoms, fragment_index=1)
                atoms_center[2]=0
                half_cell = atoms.cell[1]*.5 + atoms.cell[0]*.5
                atoms.positions += - atoms_center + half_cell
                # atoms.positions += - atoms_center + (atoms.cell[0][0]+atoms.cell[1][0])*0.5 + (atoms.cell[0][1]+atoms.cell[1][1])*0.5
                atoms.wrap()
                # atoms.positions -= half_cell
                                
                plot_atoms(atoms, ax, rotation=('0x,0y,0z'), show_unit_cell=0)
                # ax.set_title(atoms.info['adsorbate_info']['smiles'], size=8)

                ax.set_title(df_slice.material_formula.iloc[0]+' '+f"({df_slice.mi.iloc[0].replace('#','')})", size=7)
                
            ax.set_axis_off()
            # x = cell[0][0] + cell[1][0]
            # y = cell[0][1] + cell[1][1]
            ax.set_xlim((atoms.cell[0][0]+atoms.cell[1][0])*0.5-2.5, (atoms.cell[0][0]+atoms.cell[1][0])*0.5+4.5)
            ax.set_ylim((atoms.cell[0][1]+atoms.cell[1][1])*0.5-2.1, (atoms.cell[0][1]+atoms.cell[1][1])*0.5+3.8)
            
            view_atoms.append(atoms)

    fig.set_layout_engine(layout='tight')

    # fig.savefig(f"figure_S_{metal}{mi.replace('#','')}_most_stable.png", dpi=600)
    # fig.savefig(f"figure_S_{mi.replace('#','')}_most_stable.png", dpi=600)
    # fig.savefig(f"figure_S_{metal}_most_stable.png", dpi=600)
    # fig.savefig(f"figure_S_most_stable.png", dpi=600)
    
    

## figure MS

### fig2

In [ ]:
xdf = bkp_xdf.copy()
xdf = xdf[(xdf.material_formula == 'Pd') & xdf.mi.isin(['1#1#1'])]
_xdf = filter_xdf(xdf, relaxed_traj)

In [ ]:
backbone_formulas = _xdf.backbone.unique()
width_ratios = [len(_xdf[_xdf.backbone.isin([bckbone])].calibrate_keys.unique()) for bckbone in backbone_formulas] 

fig, axs = plt.subplots(1, len(backbone_formulas), figsize=[7.2, 2], sharex=False, sharey=True,
                        gridspec_kw={'width_ratios': width_ratios}
                        )

num_dict = {}

for i, bckbone in enumerate(backbone_formulas):
    
    ax = axs[i]

    pdf = _xdf[_xdf.backbone.isin([bckbone])].copy()
    pdf = pdf.sort_values(by=['calibrate_keys'])

    calibrate_keys = []
    for k in pdf.calibrate_keys.values:
        k = k.replace('H0', '')
        k = k.replace('H1', 'H')
        k = k.replace('-', '')
        for n in [str(_n) for _n in range(10)]:
            k = k.replace(n, f'$_{n}$')
        calibrate_keys.append(k)
    
    pdf['calibrate_keys'] = calibrate_keys

    heat_map = plot_energy_heatmap(pdf,
                        column = 'energy_calibrated',
                        std = 0.05,
                        e_min=-0.2,
                        e_max=1.6,
                        resolution='auto',
                        normalize=True,
                        normalize_mode='max',
                        return_heatmap=True,
                        cmap='plasma',
                        T=True,
                        ax=ax)
    
        # ax.set_title(f"{pdf.material_formula.values[0]} ({mi.replace('#', '')})", fontdict = {'fontsize': 8})
    
    ax.tick_params(top=True, labeltop=True, bottom=False, labelbottom=False)
    ax.tick_params("x", rotation=90)

    if i==0:
        ax.set_ylabel('E$_{ads}$ $_{rel}$ / eV', fontdict = {'fontsize': 7})
    num_dict[f"backbone"] = heat_map

fig.subplots_adjust(hspace=0.1, wspace=0.2)
fig.set_layout_engine(layout='tight')

In [ ]:
fig.savefig('./energy_heatmap_Cu111_max.png', dpi=900)

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=[7.2, 3], sharex=False, sharey=False, gridspec_kw={'height_ratios': [1, 2.5]}, layout='compressed')

# e_is = [0.7, 0.5, 0.5, 0.2]
e_is = [[0.,0.2,0.6,0.7],[0.,0.1,.2,.6],[0,.2,.5,.7],[0,0.2,0.4,1.]]
perplexities = [20, 10, 15,15]

plot_trj = []

for j, ck in enumerate(['C2O-H4', 'CO-H2', 'CO2-H2', 'COC-H4']):
    ax = axs[0, j]
    pdf = _xdf[_xdf.calibrate_keys.isin([ck])].copy()

    # for e in pdf.energy_calibrated.values:
    e = pdf.energy_calibrated.values
    energy_range, energy_values = get_gaussian_vectors(e, std = 0.05, e_min = -0.2, e_max = 1.6, resolution=0.001, normalize=True, normalize_mode='max')

    print(len(e[e<1.6]))
    for e_i in e[e<1.6]:
        i = find_index_by_energy(e_i, energy_range)
        ax.plot([energy_range[i], energy_range[i]], [energy_values[i], -1], linewidth=0.3, color='#ccccccff', alpha=0.6, zorder=0)
    ax.plot(energy_range, energy_values, color='#ccccccff', alpha=1,zorder=0)
    sns.scatterplot(x=energy_range, y=energy_values, hue=energy_values, alpha=1, ax=ax, legend=False, palette='plasma', markers='o', s=5, linewidth=0)
    
    # i = np.where(energy_values == np.max(energy_values))[0][0]
    # e_i = energy_values[i]
    # i = find_index_by_energy(e_i, pdf.energy_calibrated.values, _thr=0.3)
    # e_i = pdf.energy_calibrated.values[i]
    # i = find_index_by_energy(e_i, energy_range)
    # ax.plot([energy_range[i], energy_range[i]], [energy_values[i], -1], linewidth=0.5, color='black', alpha=1, zorder=10, marker='o', markersize=2)

    
    _pdf = pdf[pdf['energy_calibrated']==pdf['energy_calibrated'].min()]
    # plot_trj.append(relaxed_traj[_pdf.traj_index.values[0]])
    
    # e_i = _pdf.energy_calibrated.values[0]
    # i = find_index_by_energy(e_i, energy_range)
    # ax.plot([energy_range[i], energy_range[i]], [energy_values[i], -1], linewidth=0.5, color='black', alpha=1, zorder=10, marker='o', markersize=5)
    
    for x in e_is[j]:
        _pdf = pdf[pdf.energy_calibrated > x]
        _pdf = _pdf[_pdf['energy_calibrated']==_pdf['energy_calibrated'].min()]
        
        e_i = _pdf.energy_calibrated.values[0]
        
        smiles = _pdf.smiles.values[0]
        _a = relaxed_traj[_pdf.traj_index.values[0]].copy()
        _a.info['smiles'] = smiles
        plot_trj.append(_a)

        i = find_index_by_energy(e_i, energy_range)
        ax.plot([energy_range[i], energy_range[i]], [energy_values[i], -1], linewidth=0.5, color='black', alpha=1, zorder=20, marker='x', markersize=5, mfc='none')
    

    ax.tick_params(axis='y', which='both', length=0)
    ax.tick_params(axis='x', which='both', length=2)
    ax.tick_params(axis='x', labelsize=6)
    ax.tick_params(axis='y', labelsize=6)

    ax.set_xlabel('E$_{ads}$ $_{rel}$ / eV', fontdict = {'fontsize': 7})
    if j == 0:
        ax.set_ylabel('DOS$_{ads}$', fontdict = {'fontsize': 7})
    ax.get_yaxis().set_ticklabels([])
    ax.set_ylim([-0.01, 1.1])
    ax.set_xlim([-0.2, 1.6])

    ax = axs[1, j]
    c_trj = [relaxed_traj[i] for i in pdf.traj_index.values]    
    X_2d = get_tsne_from_traj(c_trj, perplexity=perplexities[j])
    
    x=X_2d[:,0]
    y=X_2d[:,1]
    z=pdf.energy_calibrated.values
    points = ax.scatter(x, y, c=z, s=10, cmap='jet', alpha=0.7, linewidth=0.7, vmin=0, vmax=1.6, edgecolors='black')
    
    
    ax.set_xlabel('TNSE$_{1}$', fontdict = {'fontsize': 7})
    if j == 0:
        ax.set_ylabel('TNSE$_{2}$', fontdict = {'fontsize': 7})
    ax.set_box_aspect(aspect=1)
    ax.tick_params(axis='both', which='both', length=0)
    ax.get_xaxis().set_ticklabels([])
    ax.get_yaxis().set_ticklabels([])
    

fig.subplots_adjust(hspace=0.0, wspace=0.05)
fig.set_layout_engine(layout='tight')

# cbar = fig.colorbar(points, ax=[axs[1,n] for n in [0,1,2,3]], orientation='vertical', location='right', shrink=0.6)
# cbar.set_label('E$_{ads}$ $_{rel}$ / eV', fontsize=6)
# ticklabs = cbar.ax.get_yticklabels()
# cbar.ax.set_yticklabels(ticklabs, fontsize=6)


In [ ]:
fig.savefig('./energy_heatmap_Cu111_max_pt2.png', dpi=900)

In [ ]:
for a in plot_trj[0:4]:
    print(a.info['smiles'])

In [ ]:
view(plot_trj)

In [ ]:
blend_trj = []
for atoms in plot_trj:
    a = atoms.copy()
    shift = atoms.cell[0]*0.4 + atoms.cell[1]*0.4
    cms = a[[atom.index for atom in a if a.arrays['fragments'][atom.index] > 0]].get_center_of_mass()
    cms[2] = 0
    a.positions += shift - cms
    a.wrap()
    blend_trj.append(a)

view(blend_trj)

In [ ]:
xx_trj = []
for s in ['S1SCC1O']:
    _xx = rdf[rdf.smiles==s]
    for idx in _xx.traj_index:
        xx_trj.append(relaxed_traj[idx].copy())

view(xx_trj)

In [ ]:

from autoadsorbate.utils import get_blenderized

In [ ]:
bt= get_blenderized(blend_trj, scale=[3, 1, 1], hide_spot=[0,0,0])
write('blenderized_fig2.xyz', bt)

bt= get_blenderized(blend_trj, scale=[1, 3, 1], hide_spot=[0,0,0])
write('blenderized_fig2_b.xyz', bt)

In [ ]:
view(bt)

### fig3

In [ ]:
xdf = bkp_xdf.copy()
# xdf = xdf[(xdf.material_formula == 'Cu') & xdf.mi.isin(['1#1#1','1#0#0'])]
_xdf = filter_xdf(xdf, relaxed_traj)

In [ ]:
# _xdf = bkp_xdf.copy()
mis = ['1#1#1','1#0#0', '2#1#1']#_xdf.mi.unique()
# mpids = _xdf.mpid.unique()
mpids = ['mp-30', 'mp-2']#_xdf.mpid.unique()

fig, axs = plt.subplots(len(mis), len(mpids), figsize=[7.2, 173/25.4], sharex=True, sharey=True)

num_dict = {}

for i, mpid in enumerate(mpids):
    for j, mi in enumerate(mis):

        pdf = _xdf[_xdf.mi.isin([mi]) & _xdf.mpid.isin([mpid])].copy()
        pdf = pdf.sort_values(by=['calibrate_keys'])

        calibrate_keys = []
        for k in pdf.calibrate_keys.values:
            k = k.replace('H0', '')
            k = k.replace('H1', 'H')
            k = k.replace('-', '')
            for n in [str(_n) for _n in range(10)]:
                k = k.replace(n, f'$_{n}$')
            calibrate_keys.append(k)
        pdf['calibrate_keys'] = calibrate_keys

        if len(mpids) == 1:
            ax = axs[j]
        else:
            ax = axs[j,i]
            
        
        ax.set_title(f"{pdf.material_formula.values[0]} ({mi.replace('#', '')})", fontdict = {'fontsize': 8})
        if i == 0:
            ax.set_ylabel('E$_{ads}$ $_{rel}$ / eV', fontdict = {'fontsize': 7})
        num_dict[f"{mpid}_{mi}_0"] = heat_map

fig.subplots_adjust(hspace=0.1, wspace=0.2)
fig.set_layout_engine(layout='tight')

In [ ]:
fig.savefig('./figure3.png', dpi=900)

In [ ]:
def plot_energy_heatmap_smiles(_xdf, column, std, e_min, e_max, resolution, normalize,
                        return_heatmap=False, T=False, cmap ='viridis', normalize_mode='max', ax=None):
    
    heat_map = []
    yticklabels = []
    
    for i, smiles in enumerate(_xdf.smiles.unique()):
        df_slice = _xdf[_xdf.smiles.isin([smiles])]
        if len(df_slice) > 0:
            v = energy_descriptor_from_slice(
                    df_slice,
                    column=column,
                    std=std,
                    e_min=e_min,
                    e_max=e_max,
                    resolution=resolution,
                    normalize=normalize,
                    normalize_mode=normalize_mode
                )
            heat_map.append(v[1])  
            yticklabels.append(smiles)
    heat_map = np.array(heat_map)

    xticklabels = []
    wanted_labels = np.arange(-10, 10, 0.4)
    for i, e in enumerate(v[0]):
        if any(np.abs(e - wanted_labels) < 1e-2):
            label = str(np.round(e, 1))
            if label not in xticklabels + ['-0.0']:
                xticklabels.append(label)
            else:
                xticklabels.append('')
            
        else:
            xticklabels.append('')

    if ax == None:
        fig = plt.figure()
        ax=fig.add_subplot(111)
    if T==False:
        sns.heatmap(heat_map, xticklabels=xticklabels, yticklabels=yticklabels, cbar=False, ax=ax)
        for i in range(heat_map.shape[0]+1):
            ax.axhline(i, color='white', lw=2 )
    
    else:
        ax = sns.heatmap(heat_map.T, xticklabels=yticklabels, yticklabels=xticklabels, cbar=False, cmap=cmap, ax=ax)   
        for i in range(heat_map.shape[1]+1):
            ax.axvline(i, color='white', lw=0.5)
    
    ax.tick_params(axis='both', which='both', length=0)
    ax.tick_params(axis='x', labelsize=6)
    ax.tick_params(axis='y', labelsize=6)
    ax.invert_yaxis()
    
    if return_heatmap:
        return heat_map

In [ ]:
pdf = _xdf[(_xdf.calibrate_keys == 'C2O-H4')].copy()

sort_keys = {}
for smiles in pdf.smiles.unique():
    sort_keys[smiles] = pdf[pdf.smiles==smiles].energy_calibrated.min()


pdf['sort_plot_en'] = [sort_keys[smiles] for smiles in pdf.smiles.values]
pdf = pdf.sort_values(by=['sort_plot_en'])

pdf = pdf[pdf.sort_plot_en < 1.5]

In [ ]:
len(pdf.smiles.unique())

In [ ]:
fig, ax = plt.subplots(1,1, figsize=[7.2, 2.5], sharex=False, sharey=True,
                        # gridspec_kw={'width_ratios': width_ratios}
                        )
plot_energy_heatmap_smiles(pdf.sort_values(by=['energy_calibrated']), column='energy_calibrated', std = 0.05, e_min = -0.2,
                           e_max = 1.6, resolution=0.001, normalize=True, normalize_mode='integral', return_heatmap=False, T=True, cmap ='plasma', ax = ax)

ax.set_ylabel('E$_{ads}$ $_{rel}$ / eV', fontdict = {'fontsize': 7})
fig.subplots_adjust(hspace=0.0, wspace=0.05)
fig.set_layout_engine(layout='tight')

In [ ]:
fig

In [ ]:
fig.savefig('./energy_heatmap_Cu111_max_pt3.png', dpi=900, transparent=True)

In [ ]:
from dscribe.descriptors import SOAP
from sklearn.manifold import TSNE

def get_tsne_from_traj(c_trj, perplexity=20.0):
    species = list(set([atom.symbol for atom in c_trj[0]]))
    r_cut = 6.0
    n_max = 8
    l_max = 6

    # Setting up the SOAP descriptor
    soap = SOAP(
        species=species,
        periodic=True,
        r_cut=r_cut,
        n_max=n_max,
        l_max=l_max,
    )


    soap_desc = [soap.create(a) for a in c_trj]

    tsne = TSNE(n_components=2, random_state=0, perplexity=perplexity)

    X_2d = tsne.fit_transform(np.array(soap_desc).reshape(len(soap_desc), -1))

    return X_2d

def find_index_by_energy(e_i, energy_range, _thr=0.01):
    index_candidates = np.where(np.abs(energy_range - e_i)<_thr)[0]
    while len(index_candidates) > 1:
        # print(_thr, len(index_candidates))
        _thr-=0.000001
        index_candidates = np.where(np.abs(energy_range - e_i)<_thr)[0]
    index = index_candidates[0]
    return index

e_i = e[10]
# energy_range[find_index_by_energy(e_i, energy_range)]




### figure 1 pt2

In [ ]:
import os
# import json
# import numpy as np
# from ase.io import read

path = './collect_data/'

# with open(os.path.join(path, 'identifiers.json')) as f:
#     identifiers = json.load(f)

# energy_array = np.load(os.path.join(path, 'energy_array.npy'))
# smiles = [i.split('--')[1] for i in identifiers]
# pid = [i.split('--')[0].split('elax_')[-1] for i in identifiers]
# relaxed_traj = read(os.path.join(path, 'relaxed_traj.xyz'), index=':')
parent_traj = read(os.path.join(path, 'parent_traj.xyz'), index=':')

In [ ]:
view(parent_traj)

In [ ]:
atoms = parent_traj[0].copy()
s= Surface(atoms)
s.sym_reduce()

In [ ]:
x_trj = []

for atoms in parent_traj:
    s= Surface(atoms)
    s.sym_reduce()


    for i in s.site_df.index.values:
        site_marker = s.view_site(i, return_atoms=True) 

        for atom in site_marker:
            if atom.symbol in ['Cu', 'Pd']:
                atom.symbol = 'Zn'
            if atom.symbol =='X':
                atom.symbol = 'He'
        
        x_atoms= s.atoms.copy()
        x_atoms +=site_marker
        he_cms = x_atoms[[atom.index for atom in x_atoms if atom.symbol =='He']].get_center_of_mass()
        x_atoms.positions = x_atoms.positions + atoms.cell[0]*0.5 + atoms.cell[1]*0.5 - he_cms 
        x_atoms.wrap()
        he_cms = x_atoms[[atom.index for atom in x_atoms if atom.symbol =='He']].get_center_of_mass()
        he_cms[2]=0
        x_atoms*=[3,3,1]
        x_atoms.positions -= he_cms
        
        x_trj .append(x_atoms)

view(x_trj)

In [ ]:
view(x_trj)

In [ ]:
bt = get_blenderized(x_trj, scale=[1,1,1], hide_spot=[-50,0,0])
write('sites.xyz', bt)

## MLIPX selection

In [ ]:
# import pandas as pd
# from autoadsorbate.plotting import filter_xdf

# xdf = pd.read_csv('./xdf_mlipx_full.csv')
# relaxed_traj = read('./collect_data/relaxed_traj.xyz', index=':')

# xdf = xdf[(xdf.material_formula == 'Cu')]# & xdf.mi.isin(['1#1#1','1#0#0'])]
# _xdf = filter_xdf(xdf, relaxed_traj)

# selected_trj = [relaxed_traj[i].copy() for i in _xdf[(_xdf.energy_calibrated < 0.2)].traj_index.values]
# print(len(selected_trj))

In [ ]:
# select_df = _xdf[(_xdf['C']==2) & (_xdf['O']==1) & _xdf.H.isin([4,5])]
# # select_df = _xdf[(_xdf.energy_calibrated < 300)]

# selected_trj = [relaxed_traj[i].copy() for i in select_df.traj_index.values]
# print(len(selected_trj))

In [ ]:
# selected_aads_traj = []
# all_relax = glob('/home/djrm/liac17_home/projects/aads/relax_mp-30_1#1#1_0/MACE*xyz')    

# for atoms in selected_trj:
#     uid = atoms.info['uid']
#     fs = [f for f in all_relax if len(f.split(uid))>1]
#     print(fs)
#     if len(fs)==1:
#         a = read(fs[0], index=0)
#         selected_aads_traj.append(a)
#         print('reading: ', fs[0], 'len selected: ', len(selected_aads_traj))
#     else:
#         print('Problem: ', fs, 'has len: ', len(fs))
        
    
# # /MACE_prerelax_acaef75a-2d1e-4e3a-a2d1-e4bad7d99b0a.xyz

In [ ]:
# write('mlipx_mp-30_1#1#1_C2O1H4_C2O1H5.xyz', selected_aads_traj)

In [ ]:
# view(selected_aads_traj)

## interface

In [ ]:
from hetcattoolbox.hetcattoolbox import MPResterSimple
import pymatgen
from pymatgen.analysis.interfaces.coherent_interfaces import CoherentInterfaceBuilder
from pymatgen.analysis.interfaces.zsl import ZSLGenerator

from pymatgen.io.ase import AseAtomsAdaptor

api_key = '0FZIs3w6bTqjjZdjdrc4G0wCQpKj15Ai'

mprs = MPResterSimple(api_key=api_key)

zno = AseAtomsAdaptor.get_structure(mprs.get_atoms_by_material_id('mp-2133') )
cu = AseAtomsAdaptor.get_structure(mprs.get_atoms_by_material_id('mp-30'))


zsl = ZSLGenerator(max_area=300)

cib = CoherentInterfaceBuilder(film_structure=cu,
                               substrate_structure=zno,
                               film_miller=(1,0,0),
                               substrate_miller=(1,0,0),
                               zslgen=zsl)

cib.terminations

interfaces=list(cib.get_interfaces(termination= cib.terminations[0], film_thickness = 2, substrate_thickness = 2))
# cib.terminations[0]

view_trj = [AseAtomsAdaptor.get_atoms(i) for i in interfaces]
view(view_trj)

## Cu-ZnO: Jelena Jelic paper

In [ ]:
# Cu-ZnO slab structure taken from the SI of:
#https://pubs.acs.org/doi/10.1021/acs.jpcc.4c08821

In [ ]:
zno_cu = read('./cu_zno/ZnO-Cu.poscar')
s = Surface(zno_cu)

In [ ]:
smiles = ['ClC=O', 'S1SC=[O+]1']

pop_trj = []
for smile in smiles:
    
    fragment = Fragment(smile)
    pop_trj += s.get_populated_sites(fragment, site_index='all',
                                     sample_rotation=True, mode='heuristic', 
                                     conformers_per_site_cap=5, overlap_thr=1.6, verbose=True)

for atoms in pop_trj:
    atoms.info['uid'] = str(uuid.uuid4())

write('zno_cu_ads_trj.xyz', pop_trj)


In [ ]:
import uuid
for atoms in pop_trj:
    atoms.info['uid'] = str(uuid.uuid4())

write('zno_cu_ads_trj.xyz', pop_trj)
pop_trj = read('cu_zno/zno_cu_ads_trj.xyz', index=':')


### minima hop

In [ ]:
# files = glob('/home/djrm/liac22_home/projects/aads/cu_zno/minhop/MACE*/MACE*.traj')
# frames = []
# for file in files:
#     frames+=read(file, index=':')
# print(len(files), len(frames))

In [ ]:
# fragments_map = {}
# for atoms in pop_trj:
#     fragments_map[atoms.info['uid']]=atoms.arrays['fragments']

# mh_clean_traj = []
# dists = []
# for atoms in frames:
#     a = atoms.copy()
#     a.arrays['fragments'] = fragments_map[a.info['uid']]
#     a = a[[atom.index for atom in a if a.arrays['fragments'][atom.index]>0]]
#     dist = np.max(a.get_all_distances()[0])
#     dists.append(dist)
#     if dist<1.5:
#         mh_clean_traj.append(atoms.copy())
#     # print()

In [ ]:
# write('/home/djrm/liac22_home/projects/aads/cu_zno/minhop/mh_clean_traj.xyz', mh_clean_traj)
mh_clean_traj = read('./cu_zno/mh_clean_traj_eval.xyz', index=':')

### aads relax

In [ ]:
# all_files = glob('/home/djrm/liac22_home/projects/aads/cu_zno/relax/MACE*.xyz')

# r_df, r_traj = read_relax_dir(all_files)
# _ = r_df.pop('site')
# r_df['origin'] = ['aads' for _ in r_df['uid']]

In [ ]:
# all_files = glob('/home/djrm/liac22_home/projects/aads/cu_zno/relax/MACE*.xyz')

# aads_ini_traj = []
# for file in all_files:
#     a = read(file, index = 0)

In [ ]:
# write('r_traj_relaxed.xyz', r_traj)
# r_df.to_csv('r_df_relaxed.csv')

r_traj = read('r_traj_relaxed.xyz', index=':')
r_df = pd.read_csv('r_df_relaxed.csv')

In [ ]:
from mace.calculators import mace_mp

r_df['pid'] = ['ZnO/Cu' for _ in r_df.index.values]

parent_en = {}
for pid in r_df.pid.unique():
    traj_index = r_df[r_df.pid.isin([pid])].traj_index.iloc[0]
    atoms = r_traj[traj_index].copy()
    ref_atoms = atoms[[atom.index for atom in atoms if atoms.arrays['fragments'][atom.index] == 0]]
    calc = mace_mp(model='./models/mace-mp-0b3-medium.model', dispersion=True, device='cpu')
    ref_atoms.set_calculator(calc)
    parent_en[pid] = ref_atoms.get_potential_energy()
    ref_atoms.info={'pid': pid}

ref_dict={
    'C' : 0,
    'O' : 0,
    'H' : 0
}

# parent_en = read('./relax_Cu111_Cu2_try2/MACE_relax_Cu111_Cu2_slab.xyz', index=-1).get_potential_energy()

r_xdf = compute_energy(r_df, ref_dict, parent_en)
# xdf = xdf[xdf['energy']>-100]

In [ ]:
r_xdf['energy_calibrated'] = r_xdf['energy'] - r_xdf['energy'].min()

In [ ]:
# sns.histplot(r_xdf['energy_calibrated'].values)

In [ ]:
a = r_traj[0].copy()
a = a[[atom.index for atom in a if a.arrays['fragments'][atom.index]==0]]
# s=Surface(a)

# inds = []
# for v in s.site_df.topology.values:
#     inds+=v
# inds = list(set(inds))
# plot_atoms = a[[atom.index for atom in a if atom.index in inds]]
plot_atoms = a.copy()
plot_atoms = plot_atoms*[3,3,1]
plot_atoms = sort(plot_atoms, tags=plot_atoms.positions[:,2])
# plot_atoms = sort(plot_atoms, tags=plot_atoms.arrays['fragments'])
plot_atoms.positions-=r_traj[0].cell[0]
plot_atoms.positions-=r_traj[0].cell[1]

In [ ]:
x = []
y = []
z = []
e = []

traj_cu = []
traj_interface1 = []
traj_interface2 = []
traj_zno = []

for i in r_xdf.traj_index.values:
    atoms = r_traj[i]
    atoms.wrap()
    a = atoms[[atom.index for atom in atoms if atom.symbol=='C']]
    x.append(a[0].position[0])
    y.append(a[0].position[1])
    z.append(a[0].position[2])
    energy = r_xdf.energy_calibrated.loc[i]
    e.append(energy)
    y_vals=[10,13,15.7,16.9]
    if y[-1] > y_vals[0] and y[-1] < y_vals[1]:
        traj_zno.append((atoms.copy(), energy))
    if y[-1] > y_vals[1] and y[-1] < y_vals[2]:
        traj_interface1.append((atoms.copy(), energy))
    if y[-1] > y_vals[2] and y[-1] < y_vals[3]:
        traj_interface2.append((atoms.copy(), energy))
    if y[-1] > y_vals[3]+2 and x[-1] < 3 :
        traj_cu.append((atoms.copy(), energy))
        
for t in [traj_cu, traj_interface1, traj_interface2, traj_zno]:
    t.sort(key=lambda tup: tup[1], reverse=False) 
    

x_mh = []
y_mh = []
z_mh = []
e_mh = []

e_ref = min([a.info['mace_energy'] for a in mh_clean_traj])
for atoms in mh_clean_traj:
    a = atoms[[atom.index for atom in atoms if atom.symbol=='C']]
    a.wrap()
    x_mh.append(a[0].position[0])
    y_mh.append(a[0].position[1])
    z_mh.append(a[0].position[2])
    e_mh.append(a.info['mace_energy']-e_ref)
    
x_surf = []
y_surf = []
z_surf = []
c_surf = []

for atom in plot_atoms:
    x_surf.append(atom.position[0])
    y_surf.append(atom.position[1])
    z_surf.append(atom.position[2])
    c_surf.append(atom.symbol)
    
x_pop = []
y_pop = []

for atoms in pop_trj:
    a = atoms[[atom.index for atom in atoms if atom.symbol=='C']]
    a.wrap()
    x_pop.append(a[0].position[0])
    y_pop.append(a[0].position[1])


In [ ]:
side_view_traj = [t[0][0].copy() for t in [traj_zno, traj_interface1,traj_interface2, traj_cu]]
side_view_atoms = side_view_traj[0].copy()[[]] 

for i, a in enumerate(side_view_traj):
    side_view_atoms+=a[[atom.index for atom in a if a.arrays['fragments'][atom.index] ==1]]
for atom in side_view_atoms:
    if atom.symbol == 'O':
        atom.symbol='N'
        
_a = r_traj[0].copy()
_a = a[[atom.index for atom in a if a.arrays['fragments'][atom.index]==0]]
side_view_atoms+=_a
side_view_atoms = sort(side_view_atoms, tags=side_view_atoms.positions[:,0])

view(side_view_atoms)

In [ ]:
step_count = pd.DataFrame(
    [{'origin': 'mh-md', 'runs': 9386, 'potential calls': 460380},
    {'origin': 'mh-opt', 'runs': 9880, 'potential calls': 142906},
    {'origin': 'aads', 'runs': 494, 'potential calls': 22821}]
    )
step_count

In [ ]:
read('C:\Users\user\Downloads\mace_mpa0\482\StructureOptimization')

In [ ]:
# sns.reset_orig()
# from matplotlib import interactive
# interactive(True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# sns.set_context(rc = {'patch.linewidth': 0.2})

#cbar _ stuff
_fig, _ax = plt.subplots(1, 1)
map1 = _ax.imshow(np.stack([e_mh, e_mh]),cmap='plasma')

atoms_color_dict = {
    'Cu':'#fcdbc6ff',
    'Zn':'#d6dfecff',
    'O':'#ffd6d6ff',
    'N':'#e33f3fff',
    'C':'#787878ff',
    'H':'#f6f6f6ff',
}

atoms_size_dict = {
    'Cu':600,
    'Zn':700,
    'O':300,
    'N':150,
    'C':170,
    'H':70,
    
}

fig, axs = plt.subplot_mosaic([['a','a','a','a','a','a','d','d'],
                               ['a','a','a','a','a','a','d','d'],
                               ['a','a','a','a','a','a','e','e'],
                               ['b','b','b','b','b','b','e','e'],
                               ['b','b','b','b','b','b','g','g'],
                               ['b','b','b','b','b','b','g','g'],
                               ['c','c','c','c','c','c','f','f'],
                               ['c','c','c','c','c','c','f','f'],
                               ['c','c','c','c','c','c','f','f']],
                              figsize=[180/25.4, 130/25.4])

axs['b'].sharex(axs['a'])
axs['c'].sharex(axs['a'])

# axs['d'].sharey(axs['e'])
axs['f'].sharey(axs['c'])
axs['g'].sharex(axs['e'])


for ax_label in ['a','b', 'c']:
    ax = axs[ax_label]
    y_atom = x_surf
    yrange = [-1,6]
    ax.set_ylabel(r'c / $\AA$', fontdict={'size':8})
    plot_atoms = sort(plot_atoms, tags=plot_atoms.positions[:,2])
    
    if ax_label == 'c':
        plot_atoms = sort(plot_atoms, tags=plot_atoms.positions[:,1]*-1)
        y_atom = z_surf
        yrange = [14,25]
    else:
        # ax.set_aspect('equal', adjustable='box')
        ax.set_ylabel(r'a / $\AA$', fontdict={'size':8})
    
    
    ax.set_ylim(yrange)
    ax.set_xlim([-1,23])
    ax.set_xlabel(r'b / $\AA$', fontdict={'size':8})
    
    # plt.colorbar()
    cbar = fig.colorbar(map1, ax=ax)
    cbar.set_label('E$_{ads, rel}$ / eV', rotation=90)
    
    ax.scatter(
        x=y_surf, y=y_atom, c=[atoms_color_dict[c] for c in c_surf], marker='o',
        s=[atoms_size_dict[s]*1.0 for s in c_surf], alpha=1, zorder=0, edgecolors='gray', lw=0.2
    )

ax = axs['a']
ax.scatter(x=y_pop, y=x_pop, c='k', marker='x', s=30, alpha=0.15, zorder=2)
ax.scatter(x=y, y=x, c=e, cmap='plasma', s=70, alpha=0.3, zorder=1)

ax = axs['b']
ax.scatter(x=y, y=x, c='k', marker='x', s=30, alpha=0.15, zorder=2)
ax.scatter(x=y_mh, y=x_mh, c=e_mh, cmap='plasma', s=70, alpha=0.3, zorder=1)

ax = axs['c']
# ax.scatter(x=y, y=z, c='k', marker='x', s=30, alpha=0.15, zorder=2)
ax.scatter(x=y_mh, y=z_mh, c=e_mh, cmap='plasma', s=70, alpha=0.3, zorder=1)

y_vals=[10,13,15.7,16.9]
ax.vlines(x=y_vals, ymin=[0 for _ in y_vals], ymax=[100 for _ in y_vals],
          color='k', ls='--', lw=0.7)#, label='axvline - full height')

X_loc = side_view_atoms[[atom.index for atom in side_view_atoms if atom.symbol=='C']]
ax.scatter(x=X_loc.positions[:,1],
           y=X_loc.positions[:,2],
           facecolors='none',
           edgecolors='k',
           marker='o', s=100, alpha=1, zorder=2)


ax = axs['f']
ax.scatter(
        x=side_view_atoms.positions[:,1], y=side_view_atoms.positions[:,2],
        c=[atoms_color_dict[c] for c in side_view_atoms.symbols], marker='o',
        s=[atoms_size_dict[s]*1.0 for s in side_view_atoms.symbols],
        alpha=[0.8 if _a.symbol in ['C','N', 'H'] else 0.2 for i, _a in enumerate(side_view_atoms)], zorder=0, edgecolors='k'
    )
ax.set_xlim([11,20])
ax.set_xlabel(r'b / $\AA$', fontdict={'size':8})
ax.set_ylabel(r'c / $\AA$', fontdict={'size':8})
    


sns.histplot(e_mh, ax=axs['d'], bins=20, color='orange')
sns.histplot(e, ax=axs['d'], bins=20, color='blue', alpha=0.5)
axs['d'].set_xlabel(r'E$_{ads, rel}$ / eV', fontdict={'size':8})
# axs['d'].set_xlabel(r'E$_{ads}$ / eV', fontdict={'size':8})

colors=['orange','orange','blue']
# colors=[mcolors.to_rgba(c) for c in ['#ffcb6eff', '#ffbb40ff', '#7f5d9fff']]
sns.barplot(step_count, x='origin', y='runs', ax=axs['e'], hue='origin',
            legend=False, palette=colors, edgecolor="k", alpha=0.7)
sns.barplot(step_count, x='origin', y='potential calls', ax=axs['g'], hue='origin',
            legend=False, edgecolor="k",palette=colors, alpha=0.7)
# axs['e'].set_yscale("log")
axs['g'].set_yscale("log")



fig.set_layout_engine(layout='tight')
fig.savefig(f"figure4_ms.png", dpi=600)

In [ ]:
# import chemiscope

# props = {'x': r_xdf.to_dict(orient='list')['traj_index'],
#          'e': r_xdf.to_dict(orient='list')['energy_calibrated']}
# # props = r_xdf.to_dict(orient='list')

# chemiscope.write_input(
#     # frames=[center_fragment_in_cell(r_traj[i], fragment_inds=[1]) for i in _xdf.traj_index.values],
#     frames=[r_traj[i] for i in r_xdf.traj_index.values],
#     properties=props,
#     path='chemiscope_input.json'
#     # mode='default',
# )

In [ ]:
files = glob('/home/djrm/liac22_home/projects/aads/cu_zno/minhop/MACE*/MACE*traj')
len(files)

In [ ]:
minhop_trj=[]
for file in files:
    minhop_trj+=read(file, index=':')
len(minhop_trj)

### sym reduce debug

In [6]:
from pymatgen.core import Structure
from pymatgen.io.ase import AseAtomsAdaptor
from autoadsorbate.utils import _filter_unique_sites_by_soap

structure = Structure.from_file("POSCAR")
atoms = AseAtomsAdaptor.get_atoms(structure)

s = Surface(atoms)
print(f'{len(s.site_df) = }')

xdf = _filter_unique_sites_by_soap(s.atoms, s.site_df, similarity_threshold=0.999)
print(f'{len(xdf) = }')

len(s.site_df) = 72
len(xdf) = 3
